In [ ]:
#|default_exp cli

In [ ]:
#|export
from __future__ import annotations
import warnings

from nbdev.read import *
from nbdev.sync import *
from nbdev.process import *
from nbdev.processors import *
from nbdev.doclinks import *
from nbdev.test import *
from nbdev.clean import *
from nbdev.quarto import refresh_quarto_yml

from execnb.nbio import *
from fastcore.utils import *
from fastcore.script import call_parse
from fastcore.style import S
from fastcore.shutil import rmtree,move

from os import system
from urllib.error import HTTPError
from contextlib import redirect_stdout
import os, tarfile, subprocess, sys

In [ ]:
#|hide
from nbdev import show_doc
from fastcore.test import *

# cli
> CLI commands

## Prepare -

In [ ]:
#|export
@call_parse
def prepare():
    "Export, test, and clean notebooks"
    nbdev_export.__wrapped__()
    nbdev_test.__wrapped__()
    nbdev_clean.__wrapped__()

## Filter -

In [ ]:
#|export
class FilterDefaults:
    "Override `FilterDefaults` to change which notebook processors are used"
    def xtra_procs(self): return []

    def base_procs(self):
        return [populate_language, infer_frontmatter, add_show_docs, insert_warning,
                strip_ansi, hide_line, filter_stream_, rm_header_dash,
                clean_show_doc, exec_show_docs, rm_export, clean_magics, hide_, add_links, strip_hidden_metadata]

    def procs(self):
        "Processors for export"
        return self.base_procs() + self.xtra_procs()
    
    def nb_proc(self, nb):
        "Get an `NBProcessor` with these processors"
        return NBProcessor(nb=nb, procs=self.procs())

In [ ]:
#|export
@call_parse
def nbdev_filter(
    nb_txt:str=None,  # Notebook text (uses stdin if not provided)
    fname:str=None,  # Notebook to read (uses `nb_txt` if not provided)
):
    "A notebook filter for Quarto"
    os.environ["IN_TEST"] = "1"
    try: filt = get_config().get('exporter', FilterDefaults)()
    except FileNotFoundError: filt = FilterDefaults()
    printit = False
    if fname: nb_txt = Path(fname).read_text()
    elif not nb_txt: nb_txt,printit = sys.stdin.read(),True
    nb = dict2nb(loads(nb_txt))
    if printit:
        with open(os.devnull, 'w') as dn:
            with redirect_stdout(dn): filt.nb_proc(nb).process()
    else: filt.nb_proc(nb).process()
    res = nb2str(nb)
    del os.environ["IN_TEST"]
    if printit: print(res, flush=True)
    else: return res

In [ ]:
#|hide
# print(nbdev_filter(fname='/Users/jhoward/git/nbdev/nbs/06_merge.ipynb'))

## New -

In [ ]:
#|export
def extract_tgz(url, dest='.'):
    from fastcore.net import urlopen
    with urlopen(url) as u: tarfile.open(mode='r:gz', fileobj=u).extractall(dest)

In [ ]:
#|export
def _mk_cfg(**kwargs): return {k: kwargs.get(k,None) for k in 'lib_name user branch author author_email keywords description repo'.split()}

In [ ]:
#|export
def _get_info(owner, repo, default_branch='main', default_kw='nbdev'):
    from ghapi.all import GhApi
    api = GhApi(owner=owner, repo=repo, token=os.getenv('GITHUB_TOKEN'))
    
    try: r = api.repos.get()
    except HTTPError:
        msg= [f"""Could not access repo: {owner}/{repo} to find your default branch - `{default_branch} assumed.
Edit `settings.ini` if this is incorrect.
In the future, you can allow nbdev to see private repos by setting the environment variable GITHUB_TOKEN as described here:
https://nbdev.fast.ai/cli.html#Using-nbdev_new-with-private-repos
"""]
        print(''.join(msg))
        return (default_branch,default_kw,'')
    
    return r.default_branch, default_kw if not r.topics else ' '.join(r.topics), r.description

In [ ]:
#|hide
if os.getenv('GITHUB_ACTIONS') != 'true': # GITHUB_TOKEN in actions has limited scope.
    _branch, _tags, _descrip = _get_info('fastai', 'fastai')
    test_eq(_tags, 'colab deep-learning fastai gpu machine-learning notebooks python pytorch')
    test_eq(_branch, 'master')
    test_eq(_descrip, 'The fastai deep learning library')

In [ ]:
#|export
def _fetch_from_git(raise_err=False):
    "Get information for settings.ini from the user."
    res={}
    try:
        url = run('git config --get remote.origin.url')
        res['user'],res['repo'] = repo_details(url)
        res['branch'],res['keywords'],desc = _get_info(owner=res['user'], repo=res['repo'])
        if desc: res['description'] = desc
        res['author'] = run('git config --get user.name').strip() # below two lines attempt to pull from global user config
        res['author_email'] = run('git config --get user.email').strip()
    except OSError as e:
        if raise_err: raise(e)
    else: res['lib_name'] = res['repo'].replace('-','_')
    return res

In [ ]:
#|hide
#test_eq(_fetch_from_git(raise_err=True)['lib_name'], 'nbdev')

In [ ]:
#|export
def prompt_user(cfg, inferred):
    "Let user input values not in `cfg` or `inferred`."
    print(S.dark_gray('# settings.ini'))
    res = cfg.copy()
    for k,v in cfg.items():
        inf = inferred.get(k,None)
        msg = S.light_blue(k) + ' = '
        if v is None:
            if inf is None: res[k] = input(f'# Please enter a value for {k}\n'+msg)
            else:
                res[k] = inf
                print(msg+res[k]+' # Automatically inferred from git')
        else: print(msg+str(v))
    return res

In [ ]:
#|export
def _render_nb(fn, cfg):
    "Render templated values like `{{lib_name}}` in notebook at `fn` from `cfg`"
    txt = fn.read_text()
    txt = txt.replace('from your_lib.core', f'from {cfg.lib_path}.core') # for compatibility with old templates
    for k,v in cfg.d.items(): txt = txt.replace('{{'+k+'}}', v)
    fn.write_text(txt)

In [ ]:
#|export
@call_parse
def nbdev_new(lib_name: str=None): # Package name (default: inferred from repo name)
    "Create a new project."
    from fastcore.net import urljson

    path = Path()
    tag = urljson('https://api.github.com/repos/fastai/nbdev-template/releases/latest')['tag_name']
    url = f"https://github.com/fastai/nbdev-template/archive/{tag}.tar.gz"
    extract_tgz(url)
    tmpl_path = path/f'nbdev-template-{tag}'

    defaults = _mk_cfg(lib_name=lib_name)
    inferred = _fetch_from_git()
    user_cfg = prompt_user(defaults, inferred)
    tmpl_cfg = Path(tmpl_path/'settings.ini').read_text().format(**user_cfg)
    Path('settings.ini').write_text(tmpl_cfg)
    cfg = get_config()

    nbexists = bool(first(path.glob('*.ipynb')))
    for o in tmpl_path.ls():
        if o.name == 'index.ipynb': _render_nb(o, cfg)
        if o.name == '00_core.ipynb' and not nbexists: move(str(o), './')
        elif not (path/o.name).exists(): move(str(o), './')
    rmtree(tmpl_path)

    refresh_quarto_yml()

    nbdev_export.__wrapped__()

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export() # Ensure we have the latest command below

In [ ]:
#|hide
import tempfile

In [ ]:
#|hide
# Ensure we're in an empty tempdir for testing
cwd = get_config().path('nbs_path')
try: rmtree(tmpdir)
except (NameError, FileNotFoundError): pass
tmpdir = Path(tempfile.mkdtemp())
p = tmpdir/'my-project'
p.mkdir()
os.chdir(p)

In [ ]:
#|hide
# Prepare a minimal git repo
!git init -q
!git config --local user.name fastai
!git config --local user.email info@fast.ai
!git remote add origin git@github.com:fastai/nbdev.git

Settings are inferred from the current git/GitHub repo if possible, otherwise prompted for. For example:

In [ ]:
#|hide
# NOTE: Tests below are temporarily disabled until nbdev_new can be passed required settings are command
# line args, so we can avoid calling the GitHub API which often fails in CI

In [ ]:
#|notest
!nbdev_new

# settings.ini
lib_name = nbdev # Automatically inferred from git
user = fastai # Automatically inferred from git
branch = master # Automatically inferred from git
author = fastai # Automatically inferred from git
author_email = info@fast.ai # Automatically inferred from git
keywords = conda developer-tools documentation-generator documentation-tool fastai jupyter jupyter-notebooks literate-programming nbdev pypi python python-modules # Automatically inferred from git
description = Create delightful python projects using Jupyter Notebooks # Automatically inferred from git
repo = nbdev # Automatically inferred from git


Your repo will now contain the following:

In [ ]:
%%sh
ls -a

.
..
.git
.github
.gitignore
00_core.ipynb
LICENSE
MANIFEST.in
README.md
_quarto.yml
index.ipynb
nbdev
settings.ini
setup.py
styles.css


Your information will be rendered into `index.ipynb`:

In [ ]:
#|notest
index_nb = read_nb('index.ipynb')
show_src(index_nb.cells[0].source)

```python
#| hide
from nbdev.core import *
```

In [ ]:
#|notest
show_src(index_nb.cells[1].source)

```python
# nbdev

> Create delightful python projects using Jupyter Notebooks
```

In [ ]:
#|notest
show_src(index_nb.cells[4].source.splitlines()[1], lang='sh')

```sh
pip install nbdev
```

In [ ]:
#|hide
os.chdir(cwd) # Go back to original working dir
rmtree(tmpdir)

## Help

In [ ]:
#|export
@call_parse
def chelp():
    "Show help for all console scripts"
    from fastcore.xtras import console_help
    console_help('nbdev')

In [ ]:
chelp()

nbdev_bump_version              Increment version in settings.ini by one
nbdev_changelog                 Create a CHANGELOG.md file from closed and labeled GitHub issues
nbdev_clean                     Clean all notebooks in `fname` to avoid merge conflicts
nbdev_conda                     Create a `meta.yaml` file ready to be built into a package, and optionally build and upload it
nbdev_create_config             Create a config file. Settings can be passed as command-line arguments.
nbdev_deploy                    Deploy docs to GitHub Pages
nbdev_docs                      Create Quarto docs and README.md
nbdev_export                    Export notebooks in `path` to Python modules
nbdev_filter                    A notebook filter for Quarto
nbdev_fix                       Create working notebook from conflicted notebook `nbname`
nbdev_help                      Show help for all console scripts
nbdev_install                   Install Quarto and the current library
nbdev_install_hooks  

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()